In [ ]:
# Importations
import pandas as pd
from script import pipeline_global, predire_billet, completer_margin_low_par_regression


In [ ]:
# Nouvelles données
df_nouveaux = pd.read_csv("billets.csv", sep=';')
df_nouveaux.head()


,is_genuine,diagonal,height_left,height_right,margin_low,margin_up,length
0,True,171.81,104.86,104.95,4.52,2.89,112.83
1,True,171.46,103.36,103.66,3.77,2.99,113.09
2,True,172.69,104.48,103.50,4.40,2.94,113.16
3,True,171.36,103.91,103.94,3.62,3.01,113.51
4,True,171.73,104.28,103.46,4.04,3.48,112.54


In [3]:
# Remplir les valeurs manquantes de margin_low
df_nouveaux = completer_margin_low_par_regression(df_nouveaux)

 Régression linéaire pour imputation de margin_low
R² : 0.4691
  Coefficient pour length: -0.4269
  Coefficient pour margin_up: 0.2805
  Coefficient pour height_right: 0.2745
  Intercept : 23.1826
----------------------------------------


In [ ]:
# On entraîne le modèle avec les données d’origine (ou tu peux charger un modèle sauvegardé)
best_model, scaler, _, _ = pipeline_global(df=df_nouveaux, modele_final='logreg')  


Régression Logistique:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       100
           1       0.99      0.99      0.99       200

    accuracy                           0.99       300
   macro avg       0.99      0.99      0.99       300
weighted avg       0.99      0.99      0.99       300

KNN:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       100
           1       0.99      0.99      0.99       200

    accuracy                           0.99       300
   macro avg       0.99      0.99      0.99       300
weighted avg       0.99      0.99      0.99       300

Random Forest:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       100
           1       0.99      0.99      0.99       200

    accuracy                           0.99       300
   macro avg       0.99      0.99      0.99       300
weighted avg       0.99      0.

In [6]:
# Supprimer 'is_genuine' s'il est présent (dans les données de production, il ne doit pas y être)
if 'is_genuine' in df_nouveaux.columns:
    df_nouveaux = df_nouveaux.drop(columns='is_genuine')

# Appliquer les prédictions ligne par ligne
df_nouveaux['prediction'] = df_nouveaux.apply(lambda row: predire_billet(best_model, scaler, row.to_dict()), axis=1)

# Afficher les résultats
df_nouveaux.head()
df_nouveaux['prediction'].value_counts()


prediction
VRAI billet    1001
FAUX billet     499
Name: count, dtype: int64